# Python ctypes

In [1]:
import ctypes
from ctypes import c_char, cdll

## Loading a shared library

In [2]:
libc = cdll.LoadLibrary("libc.so.6")
libc

<CDLL 'libc.so.6', handle 7f862b950000 at 0x7f8623857198>

In [3]:
import ctypes.util

name = ctypes.util.find_library('sodium')
libsodium = ctypes.cdll.LoadLibrary(name)
name, libsodium

('libsodium.so.23', <CDLL 'libsodium.so.23', handle 1a6c8b0 at 0x7f8623857da0>)

## Using shared libary functions

In [4]:
libc.random()

596516649

In [5]:
libc.printf(b"Hello, %s\n", b"World!")

14

In [6]:
len(b"Hello, World!\x00")

14

In [7]:
!python -c 'from ctypes import *; \
            libc = cdll.LoadLibrary("libc.so.6"); \
            libc.printf(b"Hello, %s\n", b"World!") '

Hello, World!


In [8]:
print(libc.time(None))

import time
print(time.time())

1548259483
1548259483.9506865


In [9]:
time.time()

1548259484.016035

## Libsodium example (digital signatures)

### Generating a new keypair

In [10]:
# allocate space for keypair
sk = bytearray(libsodium.crypto_sign_secretkeybytes())
pk = bytearray(libsodium.crypto_sign_publickeybytes())

# create corresponding ctypes
c_sk = ctypes.ARRAY(c_char, len(sk)).from_buffer(sk)
c_pk = ctypes.ARRAY(c_char, len(pk)).from_buffer(pk)

print('sk:', sk[:32].hex())
print('pk:', pk.hex())

sk: 0000000000000000000000000000000000000000000000000000000000000000
pk: 0000000000000000000000000000000000000000000000000000000000000000


In [11]:
# randomly generate a fresh keypair
libsodium.crypto_sign_keypair(c_pk, c_sk)

print('sk:', sk[:32].hex())
print('pk:', pk.hex())

sk: 638484c511c140d16ebb6cbe8be63b0d73f30a8b45e7552c5155ea206ce4135a
pk: 1be5ab00747ecac062085aafa820f1b35c95559ac214f46a2c66beb334b16808


### Signing a message

In [12]:
msg = b"Hello libsodium!"
c_msg = ctypes.create_string_buffer(msg)

sig = bytearray(libsodium.crypto_sign_bytes())
c_sig = ctypes.ARRAY(c_char, len(sig)).from_buffer(sig)
c_siglen = ctypes.c_longlong()

libsodium.crypto_sign_detached(
    c_sig, ctypes.byref(c_siglen), c_msg, len(msg), c_sk
) == 0

print(f"sig: {sig[:32].hex()}...")
print("len:", c_siglen.value)

sig: eaf34e05cc5647123f53b37ac029f6592fe1e4b23b2fbd3657be6e35ca2824ec...
len: 64


### Verifying a signature

In [13]:
libsodium.crypto_sign_verify_detached(c_sig, c_msg, len(msg), c_pk) == 0

True

In [14]:
c_other_msg = ctypes.create_string_buffer(b"Hello libsodium?")
libsodium.crypto_sign_verify_detached(
    c_sig, c_other_msg, len(c_other_msg), c_pk
) == 0

False

In [15]:
invalid_sig = sig[:]
invalid_sig[17] += 1
c_invalid_sig = ctypes.ARRAY(c_char, len(invalid_sig)).from_buffer(invalid_sig)

libsodium.crypto_sign_verify_detached(
    c_invalid_sig, c_other_msg, len(c_msg), c_pk
) == 0

False

## Writing a sharing library

In [16]:
!cat foo.h

#ifndef foo_h__
#define foo_h__
 
extern int the_answer_to_life_the_universe_and_everything(void);

#endif  // foo_h__



In [17]:
!cat foo.c

#include <stdio.h>
 
int the_answer_to_life_the_universe_and_everything(void)
{
	return 42;
}


In [18]:
!rm -f foo.o libfoo.so && ls

ctypes-demo.ipynb  foo.c  foo.h  Pipfile


In [19]:
!gcc -c -Wall -Werror -fpic foo.c

In [20]:
!gcc -shared -o libfoo.so foo.o

In [21]:
!ls

ctypes-demo.ipynb  foo.c  foo.h  foo.o	libfoo.so  Pipfile


In [22]:
libfoo = cdll.LoadLibrary('./libfoo.so')
libfoo.the_answer_to_life_the_universe_and_everything()

42